#TP3

Paulo Freitas - A100053


Pedro Santos - A100110

### Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $$N>0$$

    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
    OUTPUT r, s, t


Exercício 1

Este exercício é dirigido às provas de segurança do algoritmo acima.

  1. Construa um FOTS   $$\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$$ usando este modelo nos inteiros.
  2. Considere como propriedade de segurança:
                   safety = (r > 0) and (r < N) and (r = a*``s + b*t`)
  Prove usando k-indução que esta propriedade se verifica em qualquer traço do FOTS


  3. Prove usando “Model-Checking” com interpelantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.

  #### Nota: De momento o uso de interpolantes não é possível em `z3` e requer um dos solvers  `msat` ou `yices`. A experiência mostra que  o `pysmt`  com  Python 3.11 ou 3.12  não instala qualquer destes “solvers”;  para isso exige-se uma instalação com o Python 3.10. |

Exercício 2

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

  1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é:   $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$
  2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
  3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”.
  4. Prove a correção  do programa LPA em ambas as codificações.


# Import das Bibliotecas

Para este exercício serão usadas as bibliotecas do pysmt e os sover z3 e msat.

In [ ]:
%%capture
!yes | pip install pysmt
!apt-get install libgmp3-dev
!yes | pysmt-install --z3 --msat

file = '/usr/local/lib/python3.10/dist-packages/pysmt/smtlib/parser/__init__.py'
with open(file, 'r') as f:
    code = f.read()
    new_code = code.replace('USE_CYTHON = True', 'USE_CYTHON = False')

with open(file, 'w') as f:
  f.write(new_code)

In [ ]:
from pysmt.shortcuts import *
from pysmt.typing import INT

## EXERCÍCIO 1


### Exercício 1.1

Construa um FOTS   $$\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$$ usando este modelo nos inteiros.

In [ ]:
n=40

a=20
b=15

#mdc(a,b)

In [ ]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)

    state['r'] = Symbol('r'+str(i),INT)
    state['r_'] = Symbol('r_'+str(i),INT)
    state['s'] = Symbol('s'+str(i),INT)
    state['s_'] = Symbol('s_'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['t_'] = Symbol('t_'+str(i),INT)
    state['q'] = Symbol('q' +str(i),INT)

    return state

In [ ]:
def init(state):
    E = Equals(state['pc'],Int(0))

    R = Equals(state['r'],Int(a))
    R_ = Equals(state['r_'],Int(b))
    S = Equals(state['s'], Int(1))
    S_ = Equals(state['s_'],Int(0))
    T = Equals(state['t'],Int(0))
    T_ = Equals(state['t_'],Int(1))
    Q = Equals(state['q'],Int(0))

    return And(E,R,R_,S,S_,T,T_)

In [ ]:
def trans(curr,prox):

    t01 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r_'],Int(0))), Equals(prox['pc'],Int(1)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t02 = And(Equals(curr['pc'],Int(0)), Equals(curr['r_'],Int(0)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t10 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(0)),
              Equals(curr['q'], Div(curr['r'],curr['r_'])),
              Equals(prox['r'],curr['r_']), Equals(prox['r_'], Minus(curr['r'],Times(curr['q'],curr['r_']))),
              Equals(prox['s'],curr['s_']), Equals(prox['s_'], Minus(curr['s'],Times(curr['q'],curr['s_']))),
              Equals(prox['t'],curr['t_']), Equals(prox['t_'], Minus(curr['t'],Times(curr['q'],curr['t_']))))

    t22 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    return Or(t01,t02,t10,t22)

In [ ]:
def euclides(declare,init,trans,k):
    with Solver(name="z3") as solver:
      states = [declare(i) for i in range(k)]

      solver.add_assertion(init(states[0]))
      solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))

      if solver.solve():
        for i in range(k):
          print("Passo ",i)
          for j in states[i]:
            if j =='r':
              mdc=solver.get_value(states[i][j])
              print(j, "=", mdc,"<--")

            else:
              print(j, "=", solver.get_value(states[i][j]))
          print("--------")
        print(f'mdc({a},{b}) = {mdc}')


euclides(declare,init,trans,n)

Passo  0
pc = 0
r = 20 <--
r_ = 15
s = 1
s_ = 0
t = 0
t_ = 1
q = 1
--------
Passo  1
pc = 1
r = 20 <--
r_ = 15
s = 1
s_ = 0
t = 0
t_ = 1
q = 1
--------
Passo  2
pc = 0
r = 15 <--
r_ = 5
s = 0
s_ = 1
t = 1
t_ = -1
q = 3
--------
Passo  3
pc = 1
r = 15 <--
r_ = 5
s = 0
s_ = 1
t = 1
t_ = -1
q = 3
--------
Passo  4
pc = 0
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  5
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  6
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  7
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  8
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  9
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  10
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  11
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  12
pc = 2
r = 5 <--
r_ = 0
s = 1
s_ = -3
t = -1
t_ = 4
q = 0
--------
Passo  13

### Exercício 1.2


  2. Considere como propriedade de segurança:
                        safety = (r > 0) and (r < N) and (r = a*s + b*t)
  Prove usando k-indução que esta propriedade se verifica em qualquer traço do FOTS.

In [ ]:
def inv(state):

  p1 = GT(state['r'],Int(0))
  p2 = LT(state['r'],Int(n))
  p3 = Equals(state['r'], Plus(Times(Int(a),state['s']), Times(Int(b),state['t'])))

  return And(p1,p2,p3)

In [ ]:
def kinduction_always(declare,init,trans,inv,k):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(k)]

        solver.add_assertion(init(states[0]))

        solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))

        for i in range(k):
            solver.push()
            solver.add_assertion(Not(inv(states[i])))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                for st in states:
                    print("x, pc, inv: ", solver.get_value(st['x']), solver.get_value(st['pc']))
                return
            solver.pop()

        state2 = [declare(i+k) for i in range(k+1)]

        for i in range(k):
            solver.add_assertion(inv(state2[i]))
            solver.add_assertion(trans(state2[i],state2[i+1]))

        solver.add_assertion(Not(inv(state2[-1])))

        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            for i,state in enumerate(states):
                print(f"> State {i}: x = {solver.get_value(state['x'])}, pc= {solver.get_value(state['pc'])}.")
            return

        print(f"> A propriedade verifica-se por k-indução (k={k}).")

kinduction_always(declare,init,trans,inv,3)

> A propriedade verifica-se por k-indução (k=3).
